#Bienvenidos al taller:
# Aceleración de procesos
# en CPU/GPU
#Mini Workshop on High Performance Computing in Science and Engineering











Noviembre 2022.

# Qué veremos

En este taller, se abordará una manera (entre varias disponibles) de vectorización en CPU. Particularmente de abordará una introducción a los procesos SIMD, cuyo principio de funcionamiento y codificación se ilustrará con algunos ejemplos.

#Introducción

Procesos escalares vs vectoriales:

En este caso, los procesos escalares son aquellos que sólo pueden operar un elemento o dato a la vez.

Los procesos vectoriales, por el contrario, implementan un conjunto de instrucciones que operan en arreglos de datos, y permiten ejecutar la misma operación en multiples elementos al mismo tiempo. Este último se considera una forma de paralelismo (no concurrente) en CPU [2].

Una forma en que se puede paralelizar código en el esquema vectorial es mediante la estrategia SIMD (Single Instruction Multiple Data, por sus siglas en inglés)[1].

#Diferencia entre GPU vs CPU:

<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/renomb.jpg?raw=true:, width=800" alt="simd" width=600>

Figura 1. Generalidades entre procesos en CPU y GPU, donde en éste último se caracteriza por ejecutar procesos SIMT (Single Instructions Multiple Threads, por sus siglas en inglés).


# Introducción a SIMD

Esta basado en la idea de aplicar el mismo conjunto de instrucciones a un conjunto de elementos o datos, en contraste con los procesos escalares, como se muestra en la Figura 1:




<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/simd.png?raw=true:, width=600" alt="simd" width=600>

Figura 2: Comparación entre un proceso escalar vs un proceso SIMD. Las instrucciones (algoritmos) son el mismo para cada caso, sin embargo, dado las arquitecturas de los CPUs, es posible en el segundo caso repetir esta instrucción sobre varios conjuntos de datos al mismo tiempo. Esto dependerá del conjunto de instrucciones y la configuración de operaciones vectoriales que se tenga disponible (figura tomada de [3]).


<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/simd_proces.png?raw=true:, width=400" alt="simdpross" width=400>


Figura 3: Proceso general SIMD.


Algunos conjuntos de instrucciones comunes en procesadores actuales son SSE(X) (Streaming SIMD Extensions) y  AVX(N) (Advanced Vector Extensions, por sus siglas en inglés) , con diferentes configuraciones.




**Notas importantes del Notebook:**

Dado de que Google Colab es un entorno diseñado para Python, el cuaderno de este taller estará sujeto a los recursos disponibles que ofrece, sin tener el  rendimiento más óptimo. Las instrucciones mostradas estarán diseñadas para copilar en gcc con ayuda de las herramientas de cmake. Debido a esto, se presentarán un par de pasos extra, además de usar archivos auxiliares que se descargarán más adelante.

Recordar, que C/C++ son lenguajes copilados, es decir, el paso de la copilación genera el código a lenguaje de máquina, que a su vez origina un archivo ejecutable (cada ejemplo mostrado generará su propio archivo).





**Primera parte: Recuperación e instalación de paquetes y archivos**

Vamos a instalar las dependencias esenciales para poder copilar el proyecto:



In [ ]:
!apt-get update
!apt install -y cmake
!pip install dlib

!apt-get install git dpkg-dev binutils libx11-dev libxpm-dev libxft-dev libxext-dev
!apt-get install git tar gfortran subversion

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

Dependiendo del CPU en el que nos encontremos (tipo de procesador, arquitectura, etc) podemos saber que conjunto de instrucciones SIMD soporta mediante el siguiente comando:

In [ ]:
 %%bash
 gcc -mavx2 -dM -E - < /dev/null | egrep "SSE|AVX" | sort

#define __AVX__ 1
#define __AVX2__ 1
#define __SSE__ 1
#define __SSE2__ 1
#define __SSE2_MATH__ 1
#define __SSE3__ 1
#define __SSE4_1__ 1
#define __SSE4_2__ 1
#define __SSE_MATH__ 1
#define __SSSE3__ 1


En este caso, buscamos el conjunto de instrucciones AVX y SSE que soporta el CPU donde estamos corriendo Colab.


<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/simd-register.png?raw=true:, width=600" alt="simd" width=600>

Figura 4. Tipo de registro de vector común en CPU.

Como caso base, se suele utilizar un conjunto de instrucciones que opera con un tamaño de vector de 4 posiciones (se usará esta configuración en éste taller), ya sea entero int, flotante float o de doble precisión double (esto se puede cambiar dependiendo de las necesidades).

**Nota:** "%%bash" indica que se ejecutará una instrucción tipo shell-script (más común en entornos basados en Linux).

#Introducción a Vc y VecCore

Dentro de distintas librerias en código C/C++ que permiten trabajar con SIMD, usaremos Vc-devel [2] y Veccore [1]. Éstas librerias facilitan la codificación vectorizada, ya que son una capa de abstracción para C++. Además, proveen una API (Application Programming Interface, por sus siglas en inglés), independiente de la arquitectura del procesador y de la mayoría del conjunto de instrucciones disponibles, que permite realizar operaciones vectoriales (Portabilidad) [1,3]. Vc funciona como soporte entre el código escrito en C++ y la aplicación vectorizada (Backend), así como la aplicación en formato escalar.

Recuperamos de GitHub y extraemos entonces los archivos para la libreria de Vc:

In [ ]:
!git clone https://github.com/VcDevel/Vc.git

Cloning into 'Vc'...
remote: Enumerating objects: 37349, done.
remote: Counting objects: 100% (386/386), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 37349 (delta 231), reused 277 (delta 183), pack-reused 36963
Receiving objects: 100% (37349/37349), 11.23 MiB | 18.46 MiB/s, done.
Resolving deltas: 100% (28558/28558), done.


Usaremos las instrucciones bash para acceder a la carpeta de Vc (cd) y definir la versión adecuada (1.3.3) para el entorno de Colab y VecCore. Luego, creamos carpetas dónde copilar el proyecto (recordar que usaremos Cmake para instalar las dependencias necesarias como si se tratase de copliación en CPU directa):

In [ ]:
%%bash
cd Vc
git tag -l
git checkout -b 1.3.3 1.3.3
mkdir build
mkdir install

0.2.2
0.2.3
0.2.90
0.2.91
0.2.92
0.3.90
0.4.0
0.4.80
0.4.90
0.4.91
0.5.0
0.5.80
0.6.0
0.6.1
0.6.95
0.7.0
0.7.1
0.7.2
0.7.3
0.7.4
0.7.5
0.99.71
1.0.0
1.1.0
1.2.0
1.3.0
1.3.1
1.3.2
1.3.3
1.4.0
1.4.1
1.4.2
1.4.3


Switched to a new branch '1.3.3'


Copilaremos e instalaremos Vc en esta sesión de Colab mediante el siguiente comando de cmake usando el bash:

In [ ]:
%%bash
cd Vc
cd build
cmake -DCMAKE_INSTALL_PREFIX=/content/Vc/install  /content/Vc
make -j4
make install

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detected Compiler: GCC 7
-- MIC SDK was not found!
-- Performing Test check_cxx_compiler_flag__std_c__14
-- Performing Test check_cxx_compiler_flag__std_c__14 - Success
-- Performing Test check_c_compiler_flag__W
-- Performing Test check_c_compiler_flag__W - Success
-- Performing Test check_cxx_compiler_flag__W
-- Performing Test check_cxx_compiler_flag__W - Success
-- Performing Test check_c_compiler_flag__Wall
-- Performing Test check_c_compiler_flag__Wall - Success
-- Performing Test ch

CMake Deprecation Warning at cmake/VcMacros.cmake:35 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.
Call Stack (most recent call first):
  CMakeLists.txt:18 (include)




De igual forma, realizamos los mismos pasos para instalar VecCore (sin fijar una versión):

In [ ]:
!git clone https://gitlab.cern.ch/VecGeom/VecCore

Cloning into 'VecCore'...
remote: Enumerating objects: 4636, done.
remote: Counting objects: 100% (4636/4636), done.
remote: Compressing objects: 100% (1865/1865), done.
remote: Total 4636 (delta 2676), reused 4586 (delta 2631), pack-reused 0
Receiving objects: 100% (4636/4636), 14.54 MiB | 9.61 MiB/s, done.
Resolving deltas: 100% (2676/2676), done.


In [ ]:
%%bash
cd VecCore
mkdir build
mkdir install

Observar que fijamos como Backend la librería de Vc en las instrucciones de cmake:

In [ ]:
%%bash
cd VecCore
cd build
cmake -DCMAKE_INSTALL_PREFIX=/content/VecCore/install -DVc_DIR=/content/Vc/install/lib/cmake/Vc   \
-DVECCORE_ENABLE_VC=ON -DBACKEND=Vc -DVC=ON /content/VecCore
make -j4

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detected Compiler: GCC 7
-- Performing Test check_cxx_compiler_flag__Wabi
-- Performing Test check_cxx_compiler_flag__Wabi - Success
-- Performing Test check_cxx_compiler_flag__fabi_version_0
-- Performing Test check_cxx_compiler_flag__fabi_version_0 - Success
-- Performing Test check_cxx_compiler_flag__fabi_compat_version_0
-- Performing Test check_cxx_compiler_flag__fabi_compat_version_0 - Success
-- Performing Test check_cxx_compiler_flag__ffp_contract_fast
-- Performing Test check_cxx_

CMake Deprecation Warning at /content/Vc/install/lib/cmake/Vc/VcMacros.cmake:35 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.
Call Stack (most recent call first):
  /content/Vc/install/lib/cmake/Vc/VcConfig.cmake:17 (include)
  cmake/Builtins.cmake:14 (_find_package)
  CMakeLists.txt:37 (find_package)




In [ ]:
%%bash
cd VecCore
cd build
make install

Install the project...
-- Install configuration: ""
-- Installing: /content/VecCore/install/include/VecCore
-- Installing: /content/VecCore/install/include/VecCore/Assert.h
-- Installing: /content/VecCore/install/include/VecCore/Utilities.h
-- Installing: /content/VecCore/install/include/VecCore/Types.h
-- Installing: /content/VecCore/install/include/VecCore/CUDA.h
-- Installing: /content/VecCore/install/include/VecCore/Limits.h
-- Installing: /content/VecCore/install/include/VecCore/Backend
-- Installing: /content/VecCore/install/include/VecCore/Backend/VcVector.h
-- Installing: /content/VecCore/install/include/VecCore/Backend/VcScalar.h
-- Installing: /content/VecCore/install/include/VecCore/Backend/Scalar.h
-- Installing: /content/VecCore/install/include/VecCore/Backend/VcSimdArray.h
-- Installing: /content/VecCore/install/include/VecCore/Backend/UMESimdCommon.h
-- Installing: /content/VecCore/install/include/VecCore/Backend/UMESimd.h
-- Installing: /content/VecCore/install/include/


**Segunda parte: Programación de los ejemplos**

En esta sección, se escribirán los códigos a copilar del taller. Para ello, se emplearán estrategias de programación comunes en la vectorización de código. Particularmente, VccCore ofrece el siguiente conjunto de clases útiles:


<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/maskops.png?raw=true:, width=600" alt="simd" width=600>

Figura 5. Principales operaciones de abstracción en VecCore sobre vectores (figura tomada de [2]).



Se usará el tipo de archivo .cpp para codficar en C++, con una función main en cada ejemplo, así como los Backends tanto de Vc como escalares para ejemplificar los códigos presentados.


#Ejemplo 1

Sobreescribir la misma función tanto para un proceso escalar como uno vectorizado usando Vc de Backend (override):

Calcularemos la función ejemplo acumulada:

\begin{equation}
 \sum_{n=0}^{N} f \rightarrow
f(u_{1}, u_{2}, w) = cos(u_{1})+e^{w}+\sqrt{u_{2}}
\end{equation}

In [ ]:
%%writefile example_uno.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <chrono>
#include <cstddef>
#include <cstdint>
#include <math.h>
#include <VecCore/VecCore>
#include <Vc/Vc>
#include "VecCore/Backend/VcVector.h"

using namespace vecCore;


template<typename T>
T Counter(){
T w=0.002;
T u1= 0.9;
T u2= 12;
return math::Cos(u1)+math::Exp(w)+math::Sqrt(u2);
}


template<typename T>
void Acumulator(int n){

size_t size =  VectorSize<T>();
printf("Tamano del vector size %d \n", size);
T counter;
float count=0;
float s;


for(int i=0; i< n/size; ++i){

counter = Counter<T>();
for (int j=0; j< size ; ++j) {
s = Get(counter,j );
count = count + s;
}

}

printf("Acumulador %f \n", count);
return;
}


int main(){

printf("Primer ejemplo \n");

using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

int N=100;

auto startscalar = std::chrono::system_clock::now();
Acumulator<backend::Scalar::Float_v>(N);
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationscalar = endscalar - startscalar;
std::cout << durationscalar.count() << " ms" << std::endl;

auto startvector = std::chrono::system_clock::now();
Acumulator<backend::VcVector::Float_v>(N);
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << durationvector.count() << " ms" << std::endl;

return 0;
}

Writing example_uno.cpp


En este ejemplo se muestra el uso de los templates: template<typename T>, como estrategia para especificar el proceso escalar o vectorizado, así como el uso de la función Get de VecCore.

#Ejemplo 2

Generación de la distribución Normal mendiante la transformación de Box-Muller:

\begin{equation}
x = \mu + \sigma \sqrt{-2ln(u_{1})}  cos(2 \pi u_{2}) \rightarrow x \sim N ( \mu , \sigma )  → u_{1}, u_{2} \sim U (0,1)
\end{equation}



In [ ]:
%%writefile example_dos.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <bits/stdc++.h>
#include <random>
#include <algorithm>

#include <VecCore/VecCore>
#include <Vc/Vc>
#include <VecCore/Types.h>
#include "VecCore/Backend/VcVector.h"
#include <math.h>
#include <chrono>

using namespace vecCore;


void histograma (float *arr, int nsamples, float low, float high, int bins){
int data[bins];
int histo[bins];
float paso = (high-low)/bins;

printf("paso %f \n", paso);

float r_inf;
float r_sup;
int cont=0;
float conv_factor = 0.01*nsamples;


for (int i=0; i< bins; ++i) {

r_inf = low+paso*i;
r_sup = low+paso*(i+1);

//printf("val inf %f \n", r_inf);
//printf("val sup %f \n", r_sup);

for (int j=0; j< nsamples; ++j) {
if(   (arr[j] > r_inf )  && (arr[j]  < r_sup )      ){
cont=cont+1;
}

}
data[i]=cont;
cont=0;
}

for (int k=0; k< bins; ++k) {
printf("bins %d \n", data[k]);
}


for (int kk=0; kk< bins; ++kk) {
histo[kk] = data[kk]/conv_factor;

r_inf = low+paso*kk;
r_sup = low+paso*(kk+1);


printf("Intervalo (%f a %f ) ", r_inf, r_sup );

 for (int jj=0; jj < histo[kk]; jj++) {
  printf("*");
   }
printf(" \n");
}

return;
}



template <typename T>
T Gauss_distribution(T u1,T u2, T mean, T sigma, T pi){
T Norm=  math::Sqrt(- (math::Log(u1) + math::Log(u1)) );
return mean+sigma*(math::Cos(pi*(u2+u2))*Norm);
}


template <typename T>
void VectorGauss(int nsamples, float *array ){

size_t size =  VectorSize<T>();
printf("Tamano del vector size %d \n", size);

T res=0;
std::default_random_engine generator;
std::uniform_real_distribution<float> distribution(0.0,1.0);
T m= 0.0;
T p=3.1416;
T desv=1.0;
T numberu1;
T numberu2;

for (int i=0; i<nsamples/size ; ++i) {
numberu1 = distribution(generator);
numberu2 = distribution(generator);

res=Gauss_distribution<T>(numberu1, numberu2, m, desv, p);

for (int j=0; j< size ; ++j) {
array[i*size+j] = res[j];
}


}


return;
}



template <typename T>
void ScalarGauss(int nsamples, float* array ){

size_t size =  VectorSize<T>();
printf("size %d \n", size);

std::default_random_engine generator;
std::uniform_real_distribution<T> distribution(0.0,1.0);
T mean = 0.0;
T p=3.1416;
T desv=1.0;
T r=0;
T numberu1;
T numberu2;

for (int i=0; i<nsamples; ++i) {
numberu1 = distribution(generator);
numberu2 = distribution(generator);
array[i] = Gauss_distribution<T>(numberu1, numberu2, mean, desv, p);
}

return;
}


int main(){


int nsamples=1000000;
printf("main dos \n");

float array_scalar[nsamples];
float array_vector[nsamples];

using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

auto startscalar = std::chrono::system_clock::now();
ScalarGauss<backend::Scalar::Float_v>(nsamples, array_scalar);
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationscalar = endscalar - startscalar;
std::cout << durationscalar.count() << " ms" << std::endl;

auto startvector = std::chrono::system_clock::now();
VectorGauss<backend::VcVector::Float_v>(nsamples, array_vector);
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << durationvector.count() << " ms" << std::endl;

float bajo = -4;
float alto = 4;
int bins=14;

std::cout << "Histograma Distribucion Gaussiana Scalar" << std::endl;
histograma(array_scalar, nsamples,bajo, alto, bins );

std::cout << "Histograma Distribucion Gaussiana Vector" << std::endl;
histograma(array_vector, nsamples,bajo, alto, bins );


return 0;
}

Writing example_dos.cpp


Del ejemplo anterior, se toma la forma de sobreescribir la función que genera la distribución normal, pero se agregan dos arreglos con distribución uniforme y una función que genera un histograma sencillo. Al igual que el anterior caso, se usa la estrategia de división del vector base. El resultado final es:

<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/%20Normalvec.jpg?raw=true:, width=600" alt="norm" width=600>

Figura 6. Distribución Normal generada con 1,000,000 muestras usando una media de $\mu = 0$ y una desviación estándar de $\sigma = 1$ (generada en ROOT-CERN).

#Ejemplo 3

Cambio de dos vectores de coordenadas cartesianas $(x,y)$ a polares $( \rho , \theta ) $, generados con una distribución uniforme :

\begin{equation}
\rho = \sqrt{x^{2}+y^{2}} \qquad , \qquad \theta = atan \left( \frac{y}{x} \right)   \qquad , \qquad   x,y \sim U (0,1)
\end{equation}

In [ ]:
%%writefile example_tres.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <chrono>
#include <random>
#include <math.h>

#include <VecCore/Types.h>
#include <Vc/Vc>
#include <VecCore/VecCore>
#include "VecCore/Backend/VcVector.h"


using namespace vecCore;

template <typename T>
T Rho (T x , T y ){
return math::Sqrt(x*x+y*y);
}

template <typename T>
T Theta (T x , T y ){
return math::ATan(y/x);
}


template <typename T>
void VectorCartesian (T* Xv , T* Yv , float* R,  float* The,  size_t n){
size_t size =  VectorSize<T>();
printf("Tamano del vector size %d \n", size);
T r;
T ang;

for (int i=0; i< n ; ++i) {

r   =  Rho<T> (Xv[i] , Yv[i] );
ang =  Theta<T>(Xv[i] , Yv[i] );

for (int j=0; j< size ; ++j) {
R[i*size+j] = r[j];
The[i*size+j] = ang[j];
}

}

return;
}


template <typename T>
void ScalarCartesian (float* X , float* Y , float* R,  float* The,  int n){

size_t size =  VectorSize<T>();
printf("Tamano del vector size %d \n", size);
T r;
T ang;


for (int i=0; i< n ; ++i) {
r   =  Rho<T> (X[i] , Y[i] );
ang =  Theta<T>(X[i] , Y[i] );
R[i]=r;
The[i]=ang;
}
return;
}

int main(){

printf("Ejemplo tres \n");

int N=100000;
float array_scalar_R[N];
float array_vector_R[N];

float array_scalar_T[N];
float array_vector_T[N];

float X_scalar[N];
float Y_scalar[N];

using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

using Float_v = typename backend::VcVector::Float_v;

size_t size =  VectorSize<backend::VcVector::Float_v>();
size_t sizeV=  N/size;

Float_v X_vector[sizeV];
Float_v Y_vector[sizeV];

std::default_random_engine generator;
std::uniform_real_distribution<float> distribution(0.0,1.0);

for (int i=0; i< N; ++i) {
X_scalar[i] = distribution(generator);
Y_scalar[i] = distribution(generator);
}


for (int j=0; j < sizeV; ++j) {
for(int k=0; k < size; ++k){
Set( X_vector[j] , k ,   X_scalar[j*size+k]  );
Set( Y_vector[j] , k ,   Y_scalar[j*size+k]  );
}
}

auto startscalar = std::chrono::system_clock::now();
ScalarCartesian<backend::Scalar::Float_v>( X_scalar, Y_scalar, array_scalar_R, array_scalar_T,  N);
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationscalar = endscalar - startscalar;
std::cout << durationscalar.count() << " ms" << std::endl;

auto startvector = std::chrono::system_clock::now();
VectorCartesian<backend::VcVector::Float_v>( X_vector , Y_vector , array_vector_R, array_vector_T, sizeV );
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << durationvector.count() << " ms" << std::endl;


float r_scalar=0;
float t_scalar=0;

float r_vector=0;
float t_vector=0;

for (int l=0; l< N; ++l) {
r_scalar = r_scalar + array_scalar_R[l];
t_scalar = t_scalar + array_scalar_T[l];

r_vector = r_vector + array_vector_R[l];
t_vector = t_vector + array_vector_T[l];

}

printf("rho scalar %f theta scalar %f \n" , r_scalar, t_scalar );

printf("rho vector %f theta vector %f \n", r_vector, t_vector );

return 0;
}

Writing example_tres.cpp


En este ejemplo se ilustra una forma clásica de operar en SIMD un vector unidimensional bajo un mismo conjunto de instrucciones, así como la declaración de vector tipo Float_v y la instrucción Set de VecCore.

#Ejemplo 4

Se ilustra un caso particularmente vectorizable de la sumatoria $S$ de una función exponencial definida como:


\begin{equation}
S(x) = \sum_{n=0,k,2k,...}^{N} \sum_{i=0}^{k} \sum_{j=0}^{n} e^{-j x} → x \sim U(0,1)
\end{equation}

siendo $k$ el tamaño del vector definido por nuestro conjunto de instrucciones.


In [ ]:
%%writefile example_cuatro.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <random>
#include <cfloat>

#include <VecCore/VecCore>
#include <Vc/Vc>
#include <VecCore/Types.h>
#include "VecCore/Backend/VcVector.h"
#include <math.h>
#include <chrono>


using namespace vecCore;
using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

using Double_v = typename backend::VcVector::Double_v;

template <typename T>
T acum_function_scalar (T x , int n ){

T ac= T(0);

if(  ( x > 0)  &&  (x < 1)  ) {
for(int i=0 ; i<n ; ++i){
ac= ac +  math::Exp(-i*x);
}
}
return ac;
}

template <typename T>
T acum_function_vector (T x , int n ){

T ac= T(0);
Mask<T> gtOne (x > 0);
Mask<T> gtTwo (x < 1);
if(  MaskFull(gtOne) &&  MaskFull(gtTwo)    ) {

for(int i=0 ; i <n ; ++i){
ac= ac +  math::Exp(-i*x);
}
}
return ac;
}


template <typename T>
double VectorFun (T* vec, int N ){

size_t size  =  VectorSize<backend::VcVector::Double_v>();
size_t sizeV =  N/size;

double res=0;
T vcc = T(0);

for (int i=0; i < sizeV; ++i) {

vcc = acum_function_vector (vec[i], i);

for (int j=0; j < size; ++j) {

if(!std::isnan(vcc[j])){
res= res + vcc[j];
}else{

}

}
}

return res;

}



template <typename T>
double ScalarFun (T* vc, int N ){

size_t size =  VectorSize<backend::VcVector::Double_v>();
size_t sizeV=  N/size;

double res=0;
double r;

for (int i=0; i < sizeV; ++i) {
for(int k=0; k < size; ++k){
r= acum_function_scalar( vc[i*size+k] ,i);
if(!std::isnan(r)  ){
res= res + r;
}else{

}

}
}

return res;

}


int main(){

int N= 40000 ;
printf("Ejemplo cuatro \n");

size_t size =  VectorSize<backend::VcVector::Double_v>();
size_t sizeV=  N/size;

double array_scalar[N];
double array_vector[N];

double r_scalar;
double r_vector;

double ran_scalar[N];
Double_v ran_vector[sizeV];

std::default_random_engine generator;
std::uniform_real_distribution<double> distribution(0.0,1.0);

double r;

for (int j=0; j < sizeV; ++j) {
for(int k=0; k < size; ++k){
r = distribution(generator);
Set( ran_vector[j] , k ,   r );
ran_scalar[j*size+k] = r;
}
}

auto startscalar = std::chrono::system_clock::now();
r_scalar = ScalarFun<backend::Scalar::Double_v>(ran_scalar, N );
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration< float,std::milli> durationscalar = endscalar - startscalar;
std::cout << "Tiempo Scalar " << durationscalar.count() << " ms" << std::endl;
std::cout << "Suma Scalar  sum: " << r_scalar << std::endl;

auto startvector = std::chrono::system_clock::now();
r_vector = VectorFun<Double_v>( ran_vector,  N);
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << "Tiempo Vector " << durationvector.count() << " ms" << std::endl;
std::cout << "Suma Vector sum: " << r_vector << std::endl;


return 0;
}

Writing example_cuatro.cpp


Este ejemplo pretende reforzar la forma de manejar los datos en arreglos, ya sea dimensionales o unidemsionales, usando la estrategia SIMD. Al igual que en los anteriores ejemplos, en el entorno Colab se presenta en mayor medida el problema de retraso de datos y ejecución de las operaciones.

**Tercera parte: Copilación y ejecución de los ejemplos**

Vamos a copiar archivos importantes para la copilación desde nuestro Git:

In [ ]:
!git clone https://github.com/ChJazhiel/High_Performance_Computing_Workshop.git

Cloning into 'High_Performance_Computing_Workshop'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 95 (delta 3), reused 0 (delta 0), pack-reused 79
Unpacking objects: 100% (95/95), done.


Se copian archivos del Cmake a la carpeta raíz, en donde se ejecutará la copilación:

In [ ]:
%%bash
cp /content/High_Performance_Computing_Workshop/Aceleracion_de_procesos_en_CPU/MacroUtilities.cmake /content/
cp /content/High_Performance_Computing_Workshop/Aceleracion_de_procesos_en_CPU/CMakeLists.txt /content/

Copilamos el proyecto que contiene los ejemplos que se programaron con anterioridad usando Cmake:

In [ ]:
%%bash
cmake -DBACKEND=Vc -DVecCore_DIR=/content/VecCore/install/lib/cmake/VecCore \
-DCMAKE_BUILD_TYPE=Release -DVECCORE_ENABLE_VC=ON -DCMAKE_PREFIX_PATH=/content/Vc/install .

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring with Vc backend
-- Detected Compiler: GCC 7
-- Performing Test check_cxx_compiler_flag__Wabi
-- Performing Test check_cxx_compiler_flag__Wabi - Success
-- Performing Test check_cxx_compiler_flag__fabi_version_0
-- Performing Test check_cxx_compiler_flag__fabi_version_0 - Success
-- Performing Test check_cxx_compiler_flag__fabi_compat_version_0
-- Performing Test check_cxx_compiler_flag__fabi_compat_version_0 - Success
-- Performing Test check_cxx_compiler_flag__ffp_contract_fas

CMake Deprecation Warning at Vc/install/lib/cmake/Vc/VcMacros.cmake:35 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.
Call Stack (most recent call first):
  Vc/install/lib/cmake/Vc/VcConfig.cmake:17 (include)
  /usr/local/lib/python3.7/dist-packages/cmake/data/share/cmake-3.22/Modules/CMakeFindDependencyMacro.cmake:47 (find_package)
  VecCore/install/lib/cmake/VecCore/VecCoreConfig.cmake:62 (find_dependency)
  CMakeLists.txt:61 (find_package)


CMake Warning:
  Manually-specified variables were not used by the project:

    VECCORE_ENABLE_VC




In [ ]:
%%bash
make

[ 12%] Building CXX object CMakeFiles/returntypevec.dir/example_uno.cpp.o
[ 25%] Linking CXX executable returntypevec
[ 25%] Built target returntypevec
[ 37%] Building CXX object CMakeFiles/vectorcomplement.dir/example_dos.cpp.o
[ 50%] Linking CXX executable vectorcomplement
[ 50%] Built target vectorcomplement
[ 62%] Building CXX object CMakeFiles/vectorcartesian.dir/example_tres.cpp.o
[ 75%] Linking CXX executable vectorcartesian
[ 75%] Built target vectorcartesian
[ 87%] Building CXX object CMakeFiles/vectorsum.dir/example_cuatro.cpp.o
[100%] Linking CXX executable vectorsum
[100%] Built target vectorsum


/content/example_uno.cpp: In instantiation of ‘void Acumulator(int) [with T = float]’:
/content/example_uno.cpp:59:39:   required from here
/content/example_uno.cpp:28:7: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘size_t {aka long unsigned int}’ [-Wformat=]
 printf("Tamano del vector size %d \n", size);
 ~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/content/example_uno.cpp: In instantiation of ‘void Acumulator(int) [with T = Vc_1::Vector<float, Vc_1::VectorAbi::Sse>]’:
/content/example_uno.cpp:66:41:   required from here
/content/example_uno.cpp:28:7: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘size_t {aka long unsigned int}’ [-Wformat=]
/content/example_dos.cpp: In instantiation of ‘void ScalarGauss(int, float*) [with T = float]’:
/content/example_dos.cpp:153:61:   required from here
/content/example_dos.cpp:119:7: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘size_t {aka long unsigned

Una vez generados los ejecutables de cada ejemplo, vamos a correrlos usando el bash. El código escalar se refleja con un tamaño de vector igual a 1, mientras que el vectorizado con un tamaño de vector igual a 4. Las funciones de acumulación obtienen el mismo valor para cada tipo de ejecución, pero con tiempos de ejecución diferentes:

In [ ]:
%%bash
 ./returntypevec


Primer ejemplo 
Tamano del vector size 1 
Acumulador 508.770935 
0.010576 ms
Tamano del vector size 4 
Acumulador 508.770935 
0.006956 ms


In [ ]:
%%bash
./vectorcomplement

main dos 
size 1 
39.5643 ms
Tamano del vector size 4 
12.9421 ms
Histograma Distribucion Gaussiana Scalar
paso 0.571429 
bins 273 
bins 1861 
bins 9027 
bins 32101 
bins 83578 
bins 156956 
bins 216419 
bins 216337 
bins 157208 
bins 82892 
bins 32286 
bins 8964 
bins 1798 
bins 234 
Intervalo (-4.000000 a -3.428571 )  
Intervalo (-3.428571 a -2.857143 )  
Intervalo (-2.857143 a -2.285714 )  
Intervalo (-2.285714 a -1.714286 ) *** 
Intervalo (-1.714286 a -1.142857 ) ******** 
Intervalo (-1.142857 a -0.571428 ) *************** 
Intervalo (-0.571428 a 0.000000 ) ********************* 
Intervalo (0.000000 a 0.571429 ) ********************* 
Intervalo (0.571429 a 1.142858 ) *************** 
Intervalo (1.142858 a 1.714286 ) ******** 
Intervalo (1.714286 a 2.285715 ) *** 
Intervalo (2.285715 a 2.857143 )  
Intervalo (2.857143 a 3.428572 )  
Intervalo (3.428572 a 4.000000 )  
Histograma Distribucion Gaussiana Vector
paso 0.571429 
bins 252 
bins 1952 
bins 9200 
bins 31552 
bins 84000 
bins 

In [ ]:
%%bash
./vectorcartesian

Ejemplo tres 
Tamano del vector size 1 
2.65278 ms
Tamano del vector size 4 
2.22906 ms
rho scalar 76563.406250 theta scalar 78465.820312 
rho vector 76563.406250 theta vector 78465.820312 


In [ ]:
%%bash
./vectorsum

Ejemplo cuatro 
Tiempo Scalar 7015.93 ms
Suma Scalar  sum: 428904
Tiempo Vector 5488.46 ms
Suma Vector sum: 428904


# Muchas gracias por la atención!!

Con esto concluimos este taller, que fue impartido por Oscar Roberto Chaparro Amaro. Quedamos al pendiente para cualquier duda que presenten, dejamos el siguiente correo:
ochaparroa2019@cic.ipn.mx  , por si gustan de consultar mayor información.

<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/cic.jpeg?raw=true:, width=150" alt="simd" width=150>


**Notas finales:**

Recordar que existen varios tipos de librerias/código que trabaja con vectorización SIMD entre otras estrategias de vectorización en Harware. Lo importante es recordar y "adaptar" nuestro código a la estructura más conveniente. Hay código que aún no es posible paralelizar al 100%.

Recordar, que en este tipo de tecnologías, es necesario pensar en el Hardware disponible a la hora de códificar algoritmos.

Una fuerte ventaja de vectorización en CPUs es la independencia de Hardware y Software externo especializado (incluyendo el sistema operativo en la mayoría de los casos), como en el caso de la vectorización usando GPUs.


**(C/C++ es importante para este tipo de computación).**

#Bibliografía

[1] Speeding up software with VecCore. G Amadio, P  Canal, D Piparo, S Wenzel. J. Physics: Conf. Ser. 1085 032034, (2018).
https://iopscience.iop.org/article/10.1088/1742-6596/1085/3/032034/pdf

[2] Vc: A C++ library for explicit vectorization. M Kretz, Volker L. J. Software: Practice and Experience. Vol. 42(11), p 1409-1430, (2012).
https://onlinelibrary.wiley.com/doi/epdf/10.1002/spe.1149

[3] Portable SIMD and the VecCore Library. G Amadio, P Canal, S Wenzel. Presentation at GeantV development team (2016).
https://indico.cern.ch/event/570876/contributions/2347250/attachments/1359720/2057229/Portable-SIMD-and-the-VecCore-Library-2016-10-24.pdf

Links adicionales:


https://github.com/kokkos/kokkos/tree/master/algorithms/src

https://github.com/root-project/veccore
